# Testing the original pretrained LPRNet Model

In [15]:
!pip install onnx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 104.0 MB/s eta 0:00:00


In [ ]:
!git clone https://github.com/sirius-ai/LPRNet_Pytorch.git

fatal: destination path 'LPRNet_Pytorch' already exists and is not an empty directory.


In [ ]:
%cd LPRNet_Pytorch

/content/LPRNet_Pytorch


## Speed and Accuracy

### command to test the pretrained model

In [12]:
python test_LPRNet.py --test_img_dirs 'data/test' --pretrained_model 'weights/Final_LPRNet_model.pth'

lprnet.load_state_dict(torch.load(args.pretrained_model))
load pretrained model successful!
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
[Info] Test Accuracy: 0.901 [901:60:39:1000]
[Info] Test Speed: 0.2322s 1/1000]


In [16]:
import torch
from model.LPRNet import build_lprnet

# Load pretrained model
model = build_lprnet(lpr_max_len=8, phase=False, class_num=68, dropout_rate=0.5)
model.load_state_dict(torch.load('./weights/Final_LPRNet_model.pth', map_location = torch.device('cpu')))
model.eval()

# Export to ONNX
dummy_input = torch.randn(1, 3, 24, 94)  # Input size for LPRNet
torch.onnx.export(model, dummy_input, "./weights/Original_LPRNet.onnx", verbose=True)
print("Model exported to ONNX format.")

<ipython-input-16-0f599f939cf6>:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('./weights/Final_LPRNet_model.pth', map_location = torch.de

Model exported to ONNX format.


In [17]:
import torch
from model.LPRNet import build_lprnet

# Load pretrained model
model = build_lprnet(lpr_max_len=8, phase=False, class_num=68, dropout_rate=0.5)
model.load_state_dict(torch.load('./weights/Final_LPRNet_model.pth', map_location = torch.device('cpu')))
model.eval()

# Export to ONNX
dummy_input = torch.randn(1, 3, 24, 94)  # Input size for LPRNet
torch.onnx.export(model, dummy_input, "./weights/Original_LPRNet.onnx", verbose=True)
print("Model exported to ONNX format.")

<ipython-input-17-0f599f939cf6>:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('./weights/Final_LPRNet_model.pth', map_location = torch.de

Model exported to ONNX format.


## Size

In [18]:
# Check ONNX file size
import os
onnx_file = "./weights/Original_LPRNet.onnx"
size_mb = os.path.getsize(onnx_file) / (1024 * 1024)
print(f"ONNX Model Size: {size_mb:.2f} MB")


ONNX Model Size: 1.91 MB


## Non Zero Parameters Size

In [19]:
import onnx
import numpy as np

# Load the ONNX model
onnx_file = "./weights/Original_LPRNet.onnx"
onnx_model = onnx.load(onnx_file)

# Data type size lookup table (in bytes)
dtype_size_map = {
    onnx.TensorProto.FLOAT: 4,    # 32-bit float
    onnx.TensorProto.INT64: 8,   # 64-bit integer
    onnx.TensorProto.INT32: 4,   # 32-bit integer
    onnx.TensorProto.INT16: 2,   # 16-bit integer
    onnx.TensorProto.INT8: 1,    # 8-bit integer
    onnx.TensorProto.UINT8: 1    # 8-bit unsigned integer
}

# Calculate non-zero size
non_zero_size_bytes = 0
for initializer in onnx_model.graph.initializer:
    dtype = initializer.data_type
    dtype_size = dtype_size_map.get(dtype, 0)  # Get the size of each data type
    weights = np.frombuffer(initializer.raw_data, dtype=np.float32)  # Assume float32 by default
    non_zero_count = np.count_nonzero(weights)
    non_zero_size_bytes += non_zero_count * dtype_size

# Convert size to MB
non_zero_size_mb = non_zero_size_bytes / (1024 * 1024)
print(f"ONNX Model Size (Non-Zero Parameters Only): {non_zero_size_mb:.2f} MB")


ONNX Model Size (Non-Zero Parameters Only): 1.11 MB
